### Adding Realistic Substructure to the Lens Mass Model ###
This notebook adds more realistic perturbers in the form of sub-halos to the mass model of the lens.
First we will import all of the relevant packages and scripts.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
#setting os path to import scripts
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

#importing packages
from astropy.visualization import AsinhStretch, ImageNormalize
from astropy.visualization import simple_norm
import corner
import csv
from collections import defaultdict
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.lines as mlines
import numpy as np
from paltas.Configs.config_handler_catalog import ConfigHandler
import pickle
import statistics
from scipy.stats import multivariate_normal
from Scripts import lens_parameters, paltas_model, metrics, network_predictions, fermat_potentials
import time

### 1) Generate Sample Parameters ###
<br>
This will generate the parameters needed for generating a sample of 100 mock lens with and without substructure.

In [ ]:
f = open(module_path+'/../Data-Tables/substructure_parameters_catalog.csv', 'w+')
f.close()

#Define how many lenses will be generated in the sample
sample_num = 101

#Define how many parameters each lens has
param_num = 10

param_names = ['index','z_lens', 'gamma_md', 'theta_E_md', 'e1_md', 'e2_md', 'center_x_md', 'center_y_md', 'gamma1_md', 'gamma2_md', 'p_center_x', 'p_center_y', 'z_source', 'mag_app_source', 'R_sersic_source',
               'n_sersic_source', 'e1_source', 'e2_source', 'center_x_source', 'center_y_source', 'z_lens_light', 'mag_app_light', 'R_sersic_light', 'n_sersic_light', 'e1_light', 'e2_light', 
               'z_point_source', 'x_point_source', 'y_point_source', 'mag_app_point_source']
#Generate the parameters to be used in the sample
param_dict = lens_parameters.perturberparameters(sample_num,module_path)
#print(param_dict)

with open(module_path+'/../Data-Tables/substructure_parameters_catalog.csv', 'a') as f:
    np.savetxt(f, param_names, fmt='%s', newline=',')
    f.write('\n')
    np.savetxt(f, param_dict, fmt='%1.15f', delimiter=',')

In [ ]:
#tick = time.time()

#im_WS, metadata = paltas_model.PaltasModelWS(path,sample_num)
#im_WS = np.asarray(im_WS)

#dt = time.time() - tick
#print(sample_num," lenses took ",dt," seconds to generate.")

### 2) 100 Mock Lens Test Set With and Without Substructure ###
<br>
Here we will generate images for the network to make predictions and calculuate usefull metrics to get a better sense of how well the prediction posteriors match the known parameters.

### 2.1) Generate Images for the Network ###

In [ ]:
sample_num = 101
path = module_path+'/'
config_path = 'Configs'
image_path = 'Images_for_Network'
print(path+image_path)

In [ ]:
tick = time.time()

y_test_WoS, y_pred_WoS, std_pred_WoS, prec_pred_WoS, y_test_WS, y_pred_WS, std_pred_WS, prec_pred_WS = network_predictions.Predictions_Substructure(sample_num, path, config_path, image_path)

dt = time.time() - tick
print(sample_num," lenses took ",dt," seconds to generate.")

In [ ]:
im_WoS = []
image_path_WoS = path+image_path+'/substructure_WoS_image/'

for file in os.listdir(image_path_WoS):
    if file.endswith('.npy'):
        im_WoS.append(file)

im_WoS = np.asarray(im_WoS)
fig,axs = plt.subplots(10,10,figsize=(10,10))
norm_WoS = ImageNormalize(np.load(image_path_WoS+im_WoS[2]),stretch=AsinhStretch())

for i in range(0,sample_num-1):
    axs[i//n_cols,i%n_cols].imshow(np.load(image_path_WoS+im_WoS[i]), norm=norm_WoS)
    axs[i//n_cols,i%n_cols].set_xticks([])
    axs[i//n_cols,i%n_cols].set_yticks([])
    
plt.show()  

In [ ]:
im_WS = []
image_path_WS = path+image_path+'/substructure_WS_image/'


for file in os.listdir(image_path_WS):
    if file.endswith('.npy'):
        im_WS.append(file)

im_WS = np.asarray(im_WS)
fig,axs = plt.subplots(10,10,figsize=(10,10))
norm_WS = ImageNormalize(np.load(image_path_WS+im_WS[2]),stretch=AsinhStretch())

for i in range(0,sample_num-1):
    axs[i//n_cols,i%n_cols].imshow(np.load(image_path_WS+im_WS[i]), norm=norm_WS)
    axs[i//n_cols,i%n_cols].set_xticks([])
    axs[i//n_cols,i%n_cols].set_yticks([])
    
plt.show()  

In [ ]:
resid_norm =mcolors.TwoSlopeNorm(vmin=-0.025,vcenter=0,vmax=0.025)
fig,axs = plt.subplots(10,10,figsize=(10,10))

n_cols = 10
for i in range(0,sample_num-1):
    imris = axs[i//n_cols,i%n_cols].imshow(np.load(image_path_WS+im_WS[i])-np.load(image_path_WoS+im_WoS[i]), norm=resid_norm,cmap='bwr')
    axs[i//n_cols,i%n_cols].set_xticks([])
    axs[i//n_cols,i%n_cols].set_yticks([])

fig.colorbar(imris, ax=axs)
plt.show()
#fig.savefig(module_path+'/../Images/Sub_Res_100.png',bbox_inches='tight')

### 2.2) Calculate Metrics ###

In [ ]:
path0 = os.path.abspath(os.path.join(path+'..'))
sample_num = 101
param_num = 10
mean_metrics = metrics.PerturberSampleTrunc_Substructure(sample_num, param_num, y_test_WoS, y_pred_WoS, y_pred_WS, std_pred_WoS, std_pred_WS)
print(mean_metrics)
print(mean_metrics)
np.savetxt(path0+'/Data-Tables/substructure_metrics_base.csv', mean_metrics, fmt="%1.2f", delimiter=",")

### 3) Interpret Output from the Network ###

In [ ]:
learning_params_names = [r'$\theta_\mathrm{E}$',r'$\gamma_1$',r'$\gamma_2$',r'$\gamma_\mathrm{lens}$',r'$e_1$',
								r'$e_2$',r'$x_{lens}$',r'$y_{lens}$',r'$x_{src}$',r'$y_{src}$']

In [ ]:
param_num = 10

In [ ]:
print('y_test_WoS: ', y_test_WoS[0])
print('y_pred_WS: ', y_pred_WS[0])

In [ ]:
for i in range(sample_num-1):
    posterior_samples_WoS = multivariate_normal(mean=y_pred_WoS[i],cov=np.linalg.inv(prec_pred_WoS[i])).rvs(size=int(5e3))
    posterior_samples_WS = multivariate_normal(mean=y_pred_WS[i],cov=np.linalg.inv(prec_pred_WS[i])).rvs(size=int(5e3))

    fig = corner.corner(posterior_samples_WoS,labels=np.asarray(learning_params_names),
                        bins=20,
                show_titles=False,plot_datapoints=False,label_kwargs=dict(fontsize=30),
                levels=[0.68,0.95],color='slategray',fill_contours=True,smooth=1.0,
                hist_kwargs={'density':True,'color':'slategray','lw':3},title_fmt='.2f',max_n_ticks=3,fig=None,
                truths=y_test_WoS[i],
                truth_color='black')
    corner.corner(posterior_samples_WS,labels=np.asarray(learning_params_names),bins=20,
                show_titles=False,plot_datapoints=False,label_kwargs=dict(fontsize=30),
                levels=[0.68,0.95],color='goldenrod',fill_contours=True,smooth=1.0,
                hist_kwargs={'density':True,'color':'goldenrod','lw':3},title_fmt='.2f',max_n_ticks=3,fig=fig)

    color = ['slategray', 'goldenrod']
    label = ['Without Substructure', 'With Substructure']
    axes = np.array(fig.axes).reshape(param_num, param_num)
    axes[0,param_num-2].legend(handles=[mlines.Line2D([], [], color=color[i], label=label[i]) for i in range(0,2)],frameon=False,
                fontsize=30,loc=10)

    axes[0,3].imshow(np.load(image_path_WoS+im_WoS[i]), norm=norm_WoS)
    axes[0,4].imshow(np.load(image_path_WS+im_WS[i]), norm=norm_WS)
    #plt.show()
    plt.savefig(path0+'/Images/substructure_corner_plots/corner_plot_'+str(i)+'.png')

### 4) Fermat Potential Differences ###

### 4.1) Plotting Histograms ###

In [ ]:
#Finding image positions for WoS
x_im_test_WoS = []
y_im_test_WoS = []

for i in range(sample_num-1):
    x_im_test_val, y_im_test_val = fermat_potentials.fermat.image_positions_from_y_pred(y_test_WoS[i])
    x_im_test_WoS.append(x_im_test_val)
    y_im_test_WoS.append(y_im_test_val)

In [ ]:
file_WS = image_path_WS+'metadata.csv'
x_im_test_WS = []
y_im_test_WS = []
columns = defaultdict(list)

with open(file_WS) as file:
    csvfile = csv.reader(file)
    for row in csvfile:
        for (k,v) in enumerate(row):
            columns[k].append(v)
            
x_im_0 = columns[44]
x_im_1 = columns[45]
x_im_2 = columns[46]
x_im_3 = columns[47]

for i in range(1,sample_num):
    x_im_arr = []
    if x_im_0[i] != '':
        x_im_arr.append((x_im_0[i]))
    if x_im_1[i] != '':
        x_im_arr.append(x_im_1[i])
    if x_im_2[i] != '':
        x_im_arr.append(x_im_2[i])
    if x_im_3[i] != '':
        x_im_arr.append(x_im_3[i])
    x_im_test_WS.append(np.asarray(x_im_arr, dtype=float))

y_im_0 = columns[49]
y_im_1 = columns[50]
y_im_2 = columns[51]
y_im_3 = columns[52]

for i in range(1,sample_num):
    y_im_arr = []
    if y_im_0[i] != '':
        y_im_arr.append(y_im_0[i])
    if y_im_1[i] != '':
        y_im_arr.append(y_im_1[i])
    if y_im_2[i] != '':
        y_im_arr.append(y_im_2[i])
    if y_im_3[i] != '':
        y_im_arr.append(y_im_3[i])
    y_im_test_WS.append(np.asarray(y_im_arr, dtype=float))
    
print(x_im_test_WS)
print(y_im_test_WS)


In [ ]:
x_im_pred_WoS = []
y_im_pred_WoS = []

for i in range(sample_num-1):
    x_im_pred_val, y_im_pred_val = fermat_potentials.fermat.image_positions_from_y_pred(y_pred_WoS[i])
    x_im_pred_WoS.append(x_im_pred_val)
    y_im_pred_WoS.append(y_im_pred_val)

In [ ]:
x_im_pred_WS = []
y_im_pred_WS = []

for i in range(sample_num-1):
    x_im_pred_val, y_im_pred_val = fermat_potentials.fermat.image_positions_from_y_pred(y_pred_WS[i])
    x_im_pred_WS.append(x_im_pred_val)
    y_im_pred_WS.append(y_im_pred_val)

In [ ]:
params_dist_WoS = []
for i in range(sample_num-1):
    covariance_matrix = np.diag(std_pred_WoS[i]**2)
    params_dist_val = multivariate_normal.rvs(mean=y_pred_WoS[i],cov=covariance_matrix,size=1000)
    #plt.hist(params_dist[:,3])
    params_dist_WoS.append(params_dist_val)

In [ ]:
params_dist_WS = []
for i in range(sample_num-1):
    covariance_matrix = np.diag(std_pred_WS[i]**2)
    params_dist_val = multivariate_normal.rvs(mean=y_pred_WS[i],cov=covariance_matrix,size=1000)
    #plt.hist(params_dist[:,3])
    params_dist_WS.append(params_dist_val)

In [ ]:
# Plotting histograms
n = 0
color = ['slategray', 'goldenrod','red','black']
label = ['Without Substructure', 'With Substructure','Truth Fermat Potential Difference - WoS','Truth Fermat Potential Difference - WS']

for i in range(sample_num-1):
    truth_fermat_potentials_WoS = (fermat_potentials.fermat.fermat_potential_at_image_positions(y_test_WoS[i], x_im_test_WoS[i], y_im_test_WoS[i]))
    truth_fermat_potentials_WS = (fermat_potentials.fermat.fermat_potential_at_image_positions(y_test_WS[i], x_im_test_WS[i], y_im_test_WS[i]))
    
    #Find location of largest time delay and plot one set of hitograms for each quad
    if truth_fermat_potentials_WoS.shape == (4,):
        shape = 3
        fig,axs = plt.subplots(1,figsize=(12,4))
        im_labels = ['AB','AC','AD']
        sample_WoS_arr,sample_WS_arr,truth_arr_WoS,truth_arr_WS,abs_max_loc_WoS,abs_max_loc_WS = fermat_potentials.fermat.largest_truth_value_substructure(
        shape,params_dist_WoS[i],params_dist_WS[i],y_test_WoS[i],x_im_test_WoS[i],x_im_test_WS[i],y_im_test_WoS[i],y_im_test_WS[i],
            truth_fermat_potentials_WoS, truth_fermat_potentials_WS)
        counts,bins,_ = axs.hist(sample_WoS_arr[abs_max_loc_WoS],bins=20, histtype=u'step', color='slategray')
        counts,bins,_ = axs.hist(sample_WS_arr[abs_max_loc_WS],bins=20, histtype=u'step', color='goldenrod')
        axs.vlines(truth_arr_WoS[abs_max_loc_WoS],0,np.max(counts),zorder=200,color='red')
        axs.vlines(truth_arr_WS[abs_max_loc_WS],0,np.max(counts),zorder=200,color='black',linestyles='--')
        axs.set_title('$\Delta\phi_{'+im_labels[abs_max_loc_WoS]+'}$')
        axs.legend(handles=[mlines.Line2D([], [], color=color[i], label=label[i]) for i in range(0,4)],frameon=False, fontsize=7,loc='upper left')

    elif truth_fermat_potentials_WoS.shape == (3,):
        shape = 2
        fig,axs = plt.subplots(1,figsize=(12,4))
        im_labels = ['AB','AC']
        sample_WoS_arr,sample_WS_arr,truth_arr_WoS,truth_arr_WS,abs_max_loc_WoS,abs_max_loc_WS = fermat_potentials.fermat.largest_truth_value_substructure(
            shape,params_dist_WoS[i],params_dist_WS[i],y_test_WoS[i],x_im_test_WoS[i],x_im_test_WS[i],y_im_test_WoS[i],y_im_test_WS[i],
            truth_fermat_potentials_WoS, truth_fermat_potentials_WS)
        counts,bins,_ = axs.hist(sample_WoS_arr[abs_max_loc_WoS],bins=20, histtype=u'step', color='slategray')
        counts,bins,_ = axs.hist(sample_WS_arr[abs_max_loc_WS],bins=20, histtype=u'step', color='goldenrod')
        axs.vlines(truth_arr_WoS[abs_max_loc_WoS],0,np.max(counts),zorder=200,color='red')
        axs.vlines(truth_arr_WS[abs_max_loc_WS],0,np.max(counts),zorder=200,color='black',linestyles='--')
        axs.set_title('$\Delta\phi_{'+im_labels[abs_max_loc_WoS]+'}$')
        axs.legend(handles=[mlines.Line2D([], [], color=color[i], label=label[i]) for i in range(0,4)],frameon=False, fontsize=7,loc='upper left')

    elif truth_fermat_potentials_WoS.shape == (2,):
        fig,axs = plt.subplots(1,figsize=(12,4))
        sampled_fermat_potentials_WoS,sampled_fermat_potentials_WS = fermat_potentials.fermat.fermat_potential_arrays_substructure(
            params_dist_WoS[i],params_dist_WS[i],y_test_WoS[i],x_im_test_WoS[i],x_im_test_WS[i],y_im_test_WoS[i],y_im_test_WS[i])
        counts,bins,_ = axs.hist(sampled_fermat_potentials_WoS[:,0]-sampled_fermat_potentials_WoS[:,1],bins=20, histtype=u'step',
                                        color='slategray')
        counts,bins,_ = axs.hist(sampled_fermat_potentials_WS[:,0]-sampled_fermat_potentials_WS[:,1],bins=20, histtype=u'step',
                                        color='goldenrod')
        axs.vlines(truth_fermat_potentials_WoS[0]-truth_fermat_potentials_WoS[1],0,np.max(counts),zorder=200,color='red')
        axs.vlines(truth_fermat_potentials_WS[0]-truth_fermat_potentials_WS[1],0,np.max(counts),zorder=200,color='black',linestyles='--')
        axs.set_title('$\Delta\phi_{AB}$')
        axs.legend(handles=[mlines.Line2D([], [], color=color[i], label=label[i]) for i in range(0,4)],frameon=False,
                fontsize=7,loc='upper left')

    else:
        print('Index:'+str(i))
        print('Sampled fermat potentials WoS size: '+str(sampled_fermat_potentials_WoS[i].shape))
        print('Sampled fermat potentials WS size: '+str(sampled_fermat_potentials_WS[i].shape))
        print('Truth fermat potentials size: '+str(truth_fermat_potentials.shape))
        n = n+1

    axes[0,3].imshow(np.load(image_path_WoS+im_WoS[i]), norm=norm_WoS)
    axes[0,4].imshow(np.load(image_path_WS+im_WS[i]), norm=norm_WS)

print('Missing fermat potential differences: '+str(n))

### 4.2) Calculate Metrics ###

In [ ]:
#Accurace and Bias
acc_arr_WoS = []
acc_arr_WS = []

bias_arr_WoS = []
bias_arr_WS = []

for i in range(sample_num-1):
    truth_fermat_potentials_WoS = (fermat_potentials.fermat.fermat_potential_at_image_positions(y_test_WoS[i], x_im_test_WoS[i], y_im_test_WoS[i]))
    truth_fermat_potentials_WS = (fermat_potentials.fermat.fermat_potential_at_image_positions(y_test_WS[i], x_im_test_WS[i], y_im_test_WS[i]))

    if truth_fermat_potentials_WoS.shape == (4,):
        shape = 3
        sample_WoS_arr,sample_WS_arr,truth_arr_WoS,truth_arr_WS,abs_max_loc_WoS,abs_max_loc_WS = fermat_potentials.fermat.largest_truth_value_substructure(
            shape,params_dist_WoS[i],params_dist_WS[i],y_test_WoS[i],x_im_test_WoS[i],x_im_test_WS[i],y_im_test_WoS[i],y_im_test_WS[i],
            truth_fermat_potentials_WoS,truth_fermat_potentials_WS)
        median_WoS = statistics.median(sample_WoS_arr[abs_max_loc_WoS])
        median_WS = statistics.median(sample_WS_arr[abs_max_loc_WS])
        truth_WoS = truth_arr_WoS[abs_max_loc_WoS]
        truth_WS = truth_arr_WS[abs_max_loc_WS]
        bias_arr_WoS.append(median_WoS-truth_WoS)
        bias_arr_WS.append(median_WS-truth_WS)
        acc_arr_WoS.append(np.abs(median_WoS-truth_WoS))
        acc_arr_WS.append(np.abs(median_WS-truth_WS))

    elif truth_fermat_potentials_WoS.shape == (3,):
        shape = 2
        sample_WoS_arr,sample_WS_arr,truth_arr_WoS,truth_arr_WS,abs_max_loc_WoS,abs_max_loc_WS = fermat_potentials.fermat.largest_truth_value_substructure(
            shape,params_dist_WoS[i],params_dist_WS[i],y_test_WoS[i],x_im_test_WoS[i],x_im_test_WS[i],y_im_test_WoS[i],y_im_test_WS[i],
            truth_fermat_potentials_WoS,truth_fermat_potentials_WS)
        median_WoS = statistics.median(sample_WoS_arr[abs_max_loc_WoS])
        median_WS = statistics.median(sample_WS_arr[abs_max_loc_WS])
        truth_WoS = truth_arr_WoS[abs_max_loc_WoS]
        truth_WS = truth_arr_WS[abs_max_loc_WS]
        bias_arr_WoS.append(median_WoS-truth_WoS)
        bias_arr_WS.append(median_WS-truth_WS)
        acc_arr_WoS.append(np.abs(median_WoS-truth_WoS))
        acc_arr_WS.append(np.abs(median_WS-truth_WS))

    elif truth_fermat_potentials_WoS.shape == (2,):
        sampled_fermat_potentials_WoS,sampled_fermat_potentials_WS = fermat_potentials.fermat.fermat_potential_arrays_substructure(
            params_dist_WoS[i],params_dist_WS[i],y_test_WoS[i],x_im_test_WoS[i],x_im_test_WS[i],y_im_test_WoS[i],y_im_test_WS[i])
        sample_WoS = sampled_fermat_potentials_WoS[:,0]-sampled_fermat_potentials_WoS[:,1]
        sample_WS = sampled_fermat_potentials_WS[:,0]-sampled_fermat_potentials_WS[:,1]
        median_WoS = statistics.median(sample_WoS)
        median_WS = statistics.median(sample_WS)
        truth_WoS = truth_fermat_potentials_WoS[0]-truth_fermat_potentials_WoS[1]
        truth_WS = truth_fermat_potentials_WS[0]-truth_fermat_potentials_WS[1]
        bias_arr_WoS.append(median_WoS-truth_WoS)
        bias_arr_WS.append(median_WS-truth_WS)
        acc_arr_WoS.append(np.abs(median_WoS-truth_WoS))
        acc_arr_WS.append(np.abs(median_WS-truth_WS))

bias_metric_WoS = statistics.mean(bias_arr_WoS)
bias_metric_WS = statistics.mean(bias_arr_WS)
acc_metric_WoS = statistics.mean(acc_arr_WoS)
acc_metric_WS = statistics.mean(acc_arr_WS)
print('Accuracy of WoS: '+str(acc_metric_WoS))
print('Accuracy of WS: '+str(acc_metric_WS))
print('Bias of WoS: '+str(bias_metric_WoS))
print('Bias of WS: '+str(bias_metric_WS))

In [ ]:
#Precision

prec_arr_WoS = []
prec_arr_WS = []

for i in range(sample_num-1):
    truth_fermat_potentials_WoS = (fermat_potentials.fermat.fermat_potential_at_image_positions(y_test_WoS[i], x_im_test_WoS[i], y_im_test_WoS[i]))
    truth_fermat_potentials_WS = (fermat_potentials.fermat.fermat_potential_at_image_positions(y_test_WS[i], x_im_test_WS[i], y_im_test_WS[i]))

    if truth_fermat_potentials_WoS.shape == (4,):
        shape = 3
        sample_WoS_arr,sample_WS_arr,truth_arr_WoS,truth_arr_WS,abs_max_loc_WoS,abs_max_loc_WS = fermat_potentials.fermat.largest_truth_value_substructure(
            shape,params_dist_WoS[i],params_dist_WS[i],y_test_WoS[i],x_im_test_WoS[i],x_im_test_WS[i],y_im_test_WoS[i],y_im_test_WS[i],
            truth_fermat_potentials_WoS,truth_fermat_potentials_WS)
        std_WoS = np.std(sample_WoS_arr[abs_max_loc_WoS], ddof=1)
        std_WS = np.std(sample_WS_arr[abs_max_loc_WS], ddof=1)

        prec_arr_WoS.append(std_WoS)
        prec_arr_WS.append(std_WS)

    elif truth_fermat_potentials_WoS.shape == (3,):
        shape = 2
        sample_WoS_arr,sample_WS_arr,truth_arr_WoS,truth_arr_WS,abs_max_loc_WoS,abs_max_loc_WS = fermat_potentials.fermat.largest_truth_value_substructure(
            shape,params_dist_WoS[i],params_dist_WS[i],y_test_WoS[i],x_im_test_WoS[i],x_im_test_WS[i],y_im_test_WoS[i],y_im_test_WS[i],
            truth_fermat_potentials_WoS,truth_fermat_potentials_WS)
        std_WoS = np.std(sample_WoS_arr[abs_max_loc_WoS], ddof=1)
        std_WS = np.std(sample_WS_arr[abs_max_loc_WS], ddof=1)

        prec_arr_WoS.append(std_WoS)
        prec_arr_WS.append(std_WS)

    elif truth_fermat_potentials_WoS.shape == (2,):
        sampled_fermat_potentials_WoS,sampled_fermat_potentials_WS = fermat_potentials.fermat.fermat_potential_arrays_substructure(
            params_dist_WoS[i],params_dist_WS[i],y_test_WoS[i],x_im_test_WoS[i],x_im_test_WS[i],y_im_test_WoS[i],y_im_test_WS[i])
        sample_WoS = sampled_fermat_potentials_WoS[:,0]-sampled_fermat_potentials_WoS[:,1]
        sample_WS = sampled_fermat_potentials_WS[:,0]-sampled_fermat_potentials_WS[:,1]
        std_WoS = np.std(sample_WoS, ddof=1)
        std_WS = np.std(sample_WS, ddof=1)

        prec_arr_WoS.append(std_WoS)
        prec_arr_WS.append(std_WS)

prec_metric_WoS = statistics.mean(prec_arr_WoS)
prec_metric_WS = statistics.mean(prec_arr_WS)
print('Precision of WoS: '+str(prec_metric_WoS))
print('Precision of WS: '+str(prec_metric_WS))

In [ ]:
bias_arr_WoS = np.array(bias_arr_WoS)
bias_arr_WS = np.array(bias_arr_WS)
prec_arr_WoS = np.array(prec_arr_WoS)
prec_arr_WS = np.array(prec_arr_WS)

n_WoS = bias_arr_WoS/prec_arr_WoS
n_WS = bias_arr_WS/prec_arr_WS

n_WoS_mean = np.mean(n_WoS)
n_WS_mean = np.mean(n_WS)

_,bin_edges = np.histogram(n_WS,12)

plt.hist(n_WoS,bins=bin_edges, histtype=u'step', density=True, label="WoS: $<n_k>$=%.2f"%(n_WoS_mean))
plt.hist(n_WS,bins=bin_edges, histtype=u'step', density=True, label="WS: $<n_k>$=%.2f"%(n_WS_mean))
plt.vlines(0,0,0.5,color='black')
plt.xlabel(r'$n_k=\frac{\epsilon_k}{\sigma_k}$',fontsize=15)
plt.ylabel('Probability Density')
plt.legend()
plt.show()
plt.savefig('/Users/Logan/Documents/SJSU/HEP_Research/ComplexLA/Images/sigma_standard_error_substructure.png',bbox_inches='tight')